In [1]:
import os
import pandas as pd

# Specify the folder path and get a list of CSV files
folder_path = 'KnowledgeGraph'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file.startswith('hidy.relationships.')]

# Create an empty list to store DataFrames
dfs = []

# Iterate through each CSV file
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Replace specific values in the ':TYPE' column
merged_df[':TYPE'].replace(['cooperate', 'invest', 'same_industry', 'supply'], 1, inplace=True)
merged_df[':TYPE'].replace(['compete', 'dispute'], 0, inplace=True)

del merged_df['time']
merged_df.to_csv('merged_data.csv', index=False)

In [2]:
df_mapping = pd.read_csv('KnowledgeGraph/hidy.nodes.company.csv')

# Create mapping dictionaries
company_to_id = dict(zip(df_mapping['company_name'], df_mapping[':ID']))
id_to_company = dict(zip(df_mapping[':ID'], df_mapping['company_name']))

df_company = pd.read_excel('result_df_with_sentiment.xlsx')
df_relationships = pd.read_csv('merged_data.csv')

# Create a new column 'Explicit_ID' with company IDs
if 'Explicit_Company' in df_company.columns:
    df_company['Explicit_ID'] = df_company['Explicit_Company'].apply(lambda x: ','.join([str(company_to_id.get(name, '')) for name in x.split(', ') if name in company_to_id]))

In [3]:
df_company['Implicit_Positive_Company'] = ""
df_company['Implicit_Negative_Company'] = ""

for index, row in df_company.iterrows():
    start_id = row['Explicit_ID'].split(',')
    type_value = row['label']
    imp_pos = []
    imp_neg = []

    for i in range(len(start_id)):
        if start_id[i] != '':
            # Convert ':START_ID' column in df_relationships to integers
            df_relationships[':START_ID'] = pd.to_numeric(df_relationships[':START_ID'], errors='coerce').astype('Int64')
            matching_rows = df_relationships[df_relationships[':START_ID'] == int(start_id[i])]

            for _, relation in matching_rows.iterrows():
                end_id = relation[':END_ID']
                result = type_value == relation[':TYPE']

                if result:
                    imp_pos.append(end_id)
                else:
                    imp_neg.append(end_id)

    # Convert IDs to company names and join them into a string
    imp_pos_str = ', '.join([str(id_to_company.get(ids, '')) for ids in imp_pos])
    imp_neg_str = ', '.join([str(id_to_company.get(ids, '')) for ids in imp_neg])
    
        # Assign the resulting strings to the corresponding columns
    if imp_pos_str:
        df_company.at[index, 'Implicit_Positive_Company'] = imp_pos_str
    else:
        df_company.at[index, 'Implicit_Positive_Company'] = "None"
   
    if imp_neg_str:
        df_company.at[index, 'Implicit_Positive_Company'] = imp_neg_str
    else:
        df_company.at[index, 'Implicit_Positive_Company'] = "None"

df_company

,NewsID,NewsContent,Explicit_Company,label,Explicit_ID,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行,0,3014,"比亚迪, 中国银行, 中国铁建, 上海银行, 新华联, 中国船舶, 招商证券, 来伊份, 我...",
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,农业银行,1,2914,None,
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展, 中国国航",1,"93,3809",None,
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份,1,2878,None,
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...,冠农股份,1,2799,None,
...,...,...,...,...,...,...,...
187293,1037007,10月13日，今日共有43只涨停股，5只跌停股。其中，涨停股主要集中在华为概念股、减肥药概念...,"模塑科技, 龙版传媒, 莎普爱思, 光洋股份, 通化金马, 圣龙股份, 通宇通讯, 欧菲光",0,"542,2334,572,3460,534,3557,3220,2471","宁波华翔, 比亚迪, 长城汽车, 长鹰信质, 格力电器, 长城汽车, 新晨科技, 硕贝德, ...",
187294,1037009,吉电股份10月13日在交易所互动平台中披露，截至10月10日公司股东户数为171303户，较...,吉电股份,0,3764,"东旭蓝天, 智慧能源, 金风科技, 京能电力, 江苏索普, 智慧能源, 智慧能源",
187295,1037025,10月12日晚间，三星医疗发布2023年前三季度业绩预告，公司预计前三季度实现归属于母公司所...,三星医疗,1,,None,
187296,1037030,每经AI快讯，有投资者在投资者互动平台提问：公司领导，请问公司经营是不是出现重大问题了，股票...,亿华通,0,496,"中国船舶, 仕佳光子, 百奥泰",


In [4]:
del df_company['Explicit_ID']
df_company
df_company.to_csv('df_company.csv', index=False)

In [5]:
df_company

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行,0,"比亚迪, 中国银行, 中国铁建, 上海银行, 新华联, 中国船舶, 招商证券, 来伊份, 我...",
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,农业银行,1,None,
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展, 中国国航",1,None,
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份,1,None,
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...,冠农股份,1,None,
...,...,...,...,...,...,...
187293,1037007,10月13日，今日共有43只涨停股，5只跌停股。其中，涨停股主要集中在华为概念股、减肥药概念...,"模塑科技, 龙版传媒, 莎普爱思, 光洋股份, 通化金马, 圣龙股份, 通宇通讯, 欧菲光",0,"宁波华翔, 比亚迪, 长城汽车, 长鹰信质, 格力电器, 长城汽车, 新晨科技, 硕贝德, ...",
187294,1037009,吉电股份10月13日在交易所互动平台中披露，截至10月10日公司股东户数为171303户，较...,吉电股份,0,"东旭蓝天, 智慧能源, 金风科技, 京能电力, 江苏索普, 智慧能源, 智慧能源",
187295,1037025,10月12日晚间，三星医疗发布2023年前三季度业绩预告，公司预计前三季度实现归属于母公司所...,三星医疗,1,None,
187296,1037030,每经AI快讯，有投资者在投资者互动平台提问：公司领导，请问公司经营是不是出现重大问题了，股票...,亿华通,0,"中国船舶, 仕佳光子, 百奥泰",
